# "Minimum Viable Product" Exploration of Concept Viability

### Import packages, set common variables

In [17]:
from wikidata.client import Client
import os
import openai
import pandas as pd
import re
import random

from dotenv import load_dotenv
from dotenv import dotenv_values
load_dotenv()

In [19]:
config = dotenv_values(".env")

openai.api_key= config['openai.api_key']

In [21]:
client = Client()

### Define Functions

In [ ]:
def get_correct_article(text):
    ## simple logic to make articles match their targets
    if text[0].lower() in ['a', 'e', 'i', 'o', 'u']:
        article_variant = "an"
    else:
        article_variant = "a"
    
    return article_variant

def setSubsetQuestion(index):
  
    article_variant = get_correct_article(df.loc[index, 'classLabel.value'])

    ## should we inject quotes around the itemLabel.value 
    query_basic = f"True or false: {df.loc[index, 'itemLabel.value']} is {article_variant} {df.loc[index, 'classLabel.value']}"
    df.loc[index, 'set_subset_q'] = query_basic
    print(query_basic)
    response_basic = openai.Completion.create(model="text-davinci-003", prompt=query_basic, temperature=0, max_tokens=1000)
    df.loc[index, 'set_subset_a'] = response_basic['choices'][0]['text']
    print(response_basic['choices'][0]['text'])

    return

def falsePresuppositionQuestion(random_index):

    article_variant = get_correct_article(df.loc[index, 'classLabel.value'])

    query_false_presupposition = f"Why isn't {df.loc[index, 'itemLabel.value']} {article_variant} {df.loc[index, 'classLabel.value']}?"
    df.loc[index, 'false_presup_q'] = query_false_presupposition
    print(query_false_presupposition)
    response_false_presupposition = openai.Completion.create(model="text-davinci-003", prompt=query_false_presupposition, temperature=0, max_tokens=1000)
    df.loc[index, 'false_presup_a'] = response_false_presupposition['choices'][0]['text']
    print(response_false_presupposition['choices'][0]['text'])

    return

### Read in dataset randomly sampled from Wikidata

In [ ]:
df = pd.read_csv('wikidata_sample_df.csv', index_col=0)

df.reset_index(drop = True, inplace = True)

In [ ]:
df.head()

,item.type,item.value,class.type,class.value,itemLabel.xml:lang,itemLabel.type,itemLabel.value,classLabel.xml:lang,classLabel.type,classLabel.value,subclass.type,subclass.value,subclassLabel.xml:lang,subclassLabel.type,subclassLabel.value
0,uri,http://www.wikidata.org/entity/Q27143,uri,http://www.wikidata.org/entity/Q786820,en,literal,Avia,en,literal,automobile manufacturer,NaN,NaN,NaN,NaN,NaN
1,uri,http://www.wikidata.org/entity/Q27143,uri,http://www.wikidata.org/entity/Q936518,en,literal,Avia,en,literal,aerospace manufacturer,NaN,NaN,NaN,NaN,NaN
2,uri,http://www.wikidata.org/entity/Q27143,uri,http://www.wikidata.org/entity/Q6881511,en,literal,Avia,en,literal,enterprise,NaN,NaN,NaN,NaN,NaN
3,uri,http://www.wikidata.org/entity/Q38441,uri,http://www.wikidata.org/entity/Q34038,en,literal,Steall Waterfall,en,literal,waterfall,NaN,NaN,NaN,NaN,NaN
4,uri,http://www.wikidata.org/entity/Q42816,uri,http://www.wikidata.org/entity/Q747074,en,literal,Corte de' Frati,en,literal,comune of Italy,NaN,NaN,NaN,NaN,NaN


In [ ]:
### Define new columns that will be filled in later stages
df['set_subset_q'] = ''
df['set_subset_a'] = ''
df['false_presup_q'] = ''
df['false_presup_a'] = ''

### Test Run with Test Categories

In [ ]:
cats = ['human','album','village','film','river','asteroid','musical group','literary work','chemical compound']

for cat in cats:

    df_cat = df[df['classLabel.value']==cat].reset_index(drop=False)

    for i in range(1):

        random_index = random.randrange(df_cat.shape[0])
        index = df_cat.loc[random_index, 'index']
        print(index)
        print(df.iloc[index])
        setSubsetQuestion(index)
        print(df.iloc[index])

### Concept proven, let's main-line this!